# BonitoBook Overview

BonitoBook is a Julia-native interactive notebook system built on Bonito.jl that combines multi-language execution, AI integration, and modern web-based editing.

# Runs everywhere

  * VSCode Plot Pane
  * Browser
  * Server deployments
  * HTML displays (Documenter, Pluto)
  * Electron applications
  * JuliaHub
  * Google Colab

# Best Makie integration


In [ ]:
# Create sliders for different parameters
time_slider = Components.Slider(1:360; value=1)
spiral_factor = Components.Slider(1:50; value=20)
explosion = Components.Slider(1:100; value=50)
markersize = Components.Slider(1:100; value=30)

# Generate initial 3D galaxy data
n_points = 200
angles = LinRange(0, 4π, n_points)
radii = sqrt.(LinRange(0.1, 1, n_points)) * 8
spiral_angles = angles .+ radii * 0.3

initial_points = Point3f[]
for i in 1:n_points
    x = radii[i] * cos(spiral_angles[i]) + randn() * 0.3
    y = radii[i] * sin(spiral_angles[i]) + randn() * 0.3
    z = randn() * 2
    push!(initial_points, Point3f(x, y, z))
end

# Create figure and scatter plot
fig = Figure(backgroundcolor=:black)
ax = LScene(fig[1, 1]; show_axis=false)
splot = scatter!(ax, initial_points; color=first.(initial_points), markersize=15, transparency=true)

# JavaScript following YOUR EXACT PATTERN
jss = js"""
console.log("Initializing Galaxy...");

$(splot).then(plots=>{
    const scatter_plot = plots[0];
    const plot = scatter_plot.plot_object;
    const pos_buff = scatter_plot.geometry.attributes.positions_transformed_f32c.array;
    const initial_pos = [...pos_buff];
    console.log("Initial positions:", initial_pos.length);

    // Function to generate galaxy positions
    function generateGalaxy(timeVal, spiralVal, explosionVal) {
        const newPos = [];
        const numPoints = initial_pos.length / 3;

        for (let i = 0; i < numPoints; i++) {
            const idx = i * 3;
            const x = initial_pos[idx];
            const y = initial_pos[idx + 1];
            const z = initial_pos[idx + 2];

            // Apply time rotation
            const angle = Math.atan2(y, x) + timeVal * 0.02;
            const radius = Math.sqrt(x*x + y*y);

            // Apply spiral effect
            const spiralAngle = angle + radius * spiralVal * 0.05;

            // Apply explosion
            const scale = explosionVal / 50;

            newPos.push(
                radius * Math.cos(spiralAngle) * scale,
                radius * Math.sin(spiralAngle) * scale,
                z * scale
            );
        }
        return newPos;
    }

    // Update positions based on time slider
    $(time_slider.value).on(time_val => {
        console.log("Time:", time_val);
        const spiral = $(spiral_factor.value).value;
        const explosion = $(explosion.value).value;
        const newPos = generateGalaxy(time_val, spiral, explosion);
        plot.update([['positions_transformed_f32c', newPos]]);
    });

    // Update positions based on spiral slider
    $(spiral_factor.value).on(spiral_val => {
        console.log("Spiral:", spiral_val);
        const time = $(time_slider.value).value;
        const explosion = $(explosion.value).value;
        const newPos = generateGalaxy(time, spiral_val, explosion);
        plot.update([['positions_transformed_f32c', newPos]]);
    });

    // Update positions based on explosion slider
    $(explosion.value).on(explosion_val => {
        console.log("Explosion:", explosion_val);
        const time = $(time_slider.value).value;
        const spiral = $(spiral_factor.value).value;
        const newPos = generateGalaxy(time, spiral, explosion_val);
        plot.update([['positions_transformed_f32c', newPos]]);
    });

    // Update marker size
    $(markersize.value).on(size => {
        console.log("Size:", size);
        plot.update([['quad_scale', [size, size]], ['quad_offset', [-size/2, -size/2]]]);
    });


});
"""

# Layout
DOM.div(

    DOM.h3("🌌 3D Galaxy Explorer", style="text-align: center; color: white; margin: 10px;"),
    DOM.div(
        style="display: flex; gap: 20px; align-items: center; justify-content: center; padding: 15px; background: #1a1a2e; border-radius: 10px; margin: 10px;",
        DOM.div([DOM.label("Time: ", style="color: white; margin-right: 5px;"), time_slider]),
        DOM.div([DOM.label("Spiral: ", style="color: white; margin-right: 5px;"), spiral_factor]),
        DOM.div([DOM.label("Explosion: ", style="color: white; margin-right: 5px;"), explosion]),
        DOM.div([DOM.label("Size: ", style="color: white; margin-right: 5px;"), markersize])
    ),
    fig,
    jss,
)

# Julia native

## All components written in Julia

BonitoBook is built entirely in Julia using Bonito.jl, providing native performance and seamless integration with the Julia ecosystem.

## Supports Julia commands

```julia
]add DataFrames CSV # Package management
?println # Documentation lookup
;ls -la # Shell commands
```

# Ecosystem of Components vs Notebook

## Easy to create new components in Julia


In [ ]:
struct MyCheckbox
    value::Observable{Bool}
end

function Bonito.jsrender(session::Session, checkbox::MyCheckbox)
    return Bonito.jsrender(
        session,
        DOM.input(;
            type="checkbox",
            checked=checkbox.value,
            onchange=js"event=> $(checkbox.value).notify(event.srcElement.checked);",
        ),
    )
end
MyCheckbox(true)

## All components work standalone and can be reused


In [ ]:
using BonitoBook
BonitoBook.EvalEditor("println(\"Hello World\")\n1+1")

## Simple to create new book types with different layouts


In [ ]:
# Properly Centered Row Example
using BonitoBook
using WGLMakie  # for Row

style = Styles(
    CSS(".small-vertical .cell-editor-container",
        "width" => "200px",
        "min-width" => "0px"
    ),
    CSS(".small-vertical .cell-editor", "width" => "200px"),
    CSS(".small-vertical",
        "margin-top" => "20px",
        "margin-bottom" => "20px",
    )
)

# Create the properly centered Row
DOM.div(
    style,
    Centered(Row(
        BonitoBook.CellEditor("1+1", "julia", nothing),
        BonitoBook.CellEditor("1+2", "julia", nothing),
        width="fit-content",
        gap="50px"
    ));
    class="small-vertical"
)

## Full composability with existing Bonito apps

## Default components

### BonitoBook.Components


In [ ]:
# Create one of each component type
button = Components.Button("Submit")
slider = Components.Slider(1:100, value=50)
checkbox = Components.Checkbox(true)
dropdown = Components.Dropdown(["Option 1", "Option 2", "Option 3"], value="Option 2")
number_input = Components.NumberInput(42.0)

# Create clean layout with proper Styles
DOM.div(
    style=Styles("max-width" => "600px", "margin" => "20px auto", "padding" => "20px"),
    DOM.div(
        DOM.h3("Button"),
        button,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Slider"),
        DOM.p("Range: 1-100, Value: 50"),
        slider,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Checkbox"),
        DOM.div(checkbox, " Enabled", style=Styles("display" => "flex", "align-items" => "center")),
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Dropdown"),
        dropdown,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Number Input"),
        number_input,
        style=Styles("margin-bottom" => "20px")
    )
)


## Bonito widgets

Bonito widgets are great, but don't nicely interact with the BonitoBook theme:


In [ ]:
# Create one of each component type
button = Bonito.Button("Submit")
slider = Bonito.Slider(1:100, value=50)
checkbox = Bonito.Checkbox(true)
dropdown = Bonito.Dropdown(["Option 1", "Option 2", "Option 3"], value="Option 2")
number_input = Bonito.NumberInput(42.0)

# Create clean layout with proper Styles
DOM.div(
    style=Styles("max-width" => "600px", "margin" => "20px auto", "padding" => "20px"),
    DOM.div(
        DOM.h3("Button"),
        button,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Slider"),
        DOM.p("Range: 1-100, Value: 50"),
        slider,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Checkbox"),
        DOM.div(checkbox, " Enabled", style=Styles("display" => "flex", "align-items" => "center")),
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Dropdown"),
        dropdown,
        style=Styles("margin-bottom" => "20px")
    ),
    DOM.div(
        DOM.h3("Number Input"),
        number_input,
        style=Styles("margin-bottom" => "20px")
    )
)


### @manipulate


In [ ]:
import Makie.SpecApi as S
funcs = (sqrt=sqrt, x_square=x->x^2, sin=sin, cos=cos)
colormaps = ["viridis", "heat", "blues"]
types = (scatter=S.Scatter, lines=S.Lines, linesegments=S.LineSegments)
sizes=10:0.1:100
checkbox = (true, false)
@manipulate for cmap=colormaps, func=funcs, Typ=types, size=sizes, show_legend=checkbox
    x = 0:0.3:10
    s = Typ == S.Scatter ? (; markersize=size) : (; linewidth=size)
    splot = Typ(x, func.(x); colormap=cmap, color=x, s...)
    ax = S.Axis(; plots=[splot])
    if show_legend
        cbar = S.Colorbar(splot)
        S.GridLayout([ax cbar])
    else
        S.GridLayout([ax])
    end
end

### LaTeX support

```latex
$$\int_{-\infty}^{\infty} e^{-x^2} dx = \sqrt{\pi}$$

$$\mathbf{A} = \begin{pmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{pmatrix}$$
```

# Python integration

## Package management


In [ ]:
]add numpy matplotlib pandas

## Shared namespace


In [ ]:
import numpy as np
data = np.random.randn(1000, 2)
labels = ["x", "y"]

In [ ]:
using WGLMakie
# Access Python variables directly in Julia
scatter(data[:, 1], data[:, 2], axis=(xlabel=labels[1], ylabel=labels[2]))

## Rich MIME support


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot([1, 2, 3, 4], [1, 4, 2, 3])
fig # Automatically displays in notebook

## @edit support

```julia
BonitoBook.@bedit println("hello") # Opens function source in editor
```

## Revise.jl integration

Changes from e.g. `@edit` are automatically applied.

## Style customization

Edit `styles/style.jl` by pressing the paintcan icon to customize appearance:


In [ ]:
# Modify colors, fonts, layout dimensions
light_theme = true # Force light theme
editor_width = "800px" # Adjust editor width;

# Export/import options

## Import formats

### Jupyter notebooks (.ipynb)

### Markdown files (.md)

## Export formats

### HTML export

### Markdown export

### Julia script export

## Folder structure

Each book creates a structured project:

```
mybook/
├── Project.toml         # Julia dependencies
├── Manifest.toml        # Dependency lock file
├── book.md              # Main content
├── styles/style.jl      # Custom styling
├── ai/
│   ├── config.toml      # AI configuration
│   └── system-prompt.md # Custom AI prompt
├── data/               # Data files
└── .versions/          # Automatic backups
```

This folder can be zipped and shared with all data, settings and style. With Project.toml and Manifest being part of the format, each notebook is reproducable.

# Advanced features

## Multi-language cells

Books support mixing Julia, Python, and Markdown cells seamlessly with shared variable namespaces.

## Asynchronous execution

Code runs in background threads without blocking the UI. Multiple cells can execute concurrently.

## Automatic backups

All changes are automatically saved to `.versions/` with timestamps for version recovery.

## Responsive design

UI adapts to different screen sizes and orientations. Works on desktop, tablet, and mobile browsers.

## Theme system

Automatic dark/light mode detection with manual override support. Consistent theming across all components.

## Sidebar system

Collapsible sidebars for tools, file browser, chat, and custom widgets. Configurable positioning and behavior.

## Live reloading

Files are watched for changes and automatically reloaded. Useful for development workflows with external editors.
